# MinHASH LSH

wyszukiwanie podobnych dokumentow i duplukatow uzywajac wyjątkowo małej reprezentacji dokumentów

In [1]:
# Importujemy pyspark oraz tworzyny nowy kontekst (reprezentujący połączenie z serwerem spark).
# W tym przypadku serwer zostanie wystartowny automatycznie.

import pyspark 
import pyspark.sql as psql

sc = pyspark.SparkContext() # SparkContext to Entry Point do jakiejkolwiek funkcjonalności spark'a.
spark_session = psql.SparkSession(sc) # Główny Entry Point do DataFrame i funkcjonalności SQL

Pierwszym krokiem jest tokenizacja [link](https://nlp.stanford.edu/IR-book/html/htmledition/tokenization-1.html]). Zrobimy to na zabawkowym przykładzie ale z wykorzystaniem rozproszenia oferowanego przez spark.

In [3]:
from pyspark.sql.functions import udf, col # User-defined functions | Column expression

# Zbiór dokumentów jako DataFrame, a każdy wiersz to bag-of-words

raw_docs = spark_session.createDataFrame([
    ("Ala ma kota a kot ma ale", ),
    ("Hania ma kangura ale nie ma kota", ),
    ("A to jest zupelnie inne zdanie", )
], ["text"])

raw_docs.show()

tokenizer = udf(lambda text: text.split()) # Stwórzmy tokenizer i zdefinujmy go jako User-defined function

# Nowa kolumna na podstawie 
tokenized_docs = raw_docs.withColumn('tokenized', tokenizer(col('text')))

tokenized_docs.show()
print (tokenized_docs.first())

+--------------------+
|                text|
+--------------------+
|Ala ma kota a kot...|
|Hania ma kangura ...|
|A to jest zupelni...|
+--------------------+

+--------------------+--------------------+
|                text|           tokenized|
+--------------------+--------------------+
|Ala ma kota a kot...|[Ala, ma, kota, a...|
|Hania ma kangura ...|[Hania, ma, kangu...|
|A to jest zupelni...|[A, to, jest, zup...|
+--------------------+--------------------+

Row(text='Ala ma kota a kot ma ale', tokenized='[Ala, ma, kota, a, kot, ma, ale]')


### Zadanie 1
wykorzystaj gotowy tokenizer z biblioteki ml.feature

In [4]:
import pyspark.ml.feature as feature

tokenizer = feature.Tokenizer(inputCol="text", outputCol="tokenized")
# Tokenizacja to rozdzielanie tekstu na poszczególne terminy (słowa)

tokenized_docs = tokenizer.transform(raw_docs)

tokenized_docs.show()

# Przkład, na bazie którego się uczyłem (zakładka Python):
# https://spark.apache.org/docs/latest//ml-features.html#tokenizer

+--------------------+--------------------+
|                text|           tokenized|
+--------------------+--------------------+
|Ala ma kota a kot...|[ala, ma, kota, a...|
|Hania ma kangura ...|[hania, ma, kangu...|
|A to jest zupelni...|[a, to, jest, zup...|
+--------------------+--------------------+



Tak jak na zajęciach WDAM, pracowaliśmy z wektorową reprezentacją dokumentów [Vector-Space Model](https://en.wikipedia.org/wiki/Vector_space_model) tutaj zrobimy to samo przy uzyciu sparkowej biblioteki do uczenia maszynkowego (mllib), lecz zamiast znanego nam modelu TF-IDF ( _Term frequency-inverse document frequency_ ) uzyjemy skip-n-gram modelu [link](https://towardsdatascience.com/skip-gram-nlp-context-words-prediction-algorithm-5bbf34f84e0c) zaimplementowanego w obiekcie Word2Vec (Word2Vec to estymator, który bierze sekwencje słów reprezentujących dokumenty i trenuje Word2VecModel). Odsylam do dokumentacji [tutaj](https://spark.apache.org/docs/2.2.0/mllib-feature-extraction.html).

In [5]:
from pyspark.mllib.feature import Word2Vec

# Stworzenie Resilient Distributed Datasets
rdd = sc.parallelize( [
    "Ala ma kota a kot ma ale".lower().split(' '),
    "Hania ma kangura ale nie ma kota".lower().split(' '), 
    "A to jest zupelnie inne zdanie".lower().split(' ') ])
# Możemy użyć pre-trenowanego modelu, ale w kontekście naszego problemu o wiele lepiej znaleźć parametry modelu samemu
model = Word2Vec().setVectorSize(4).setMinCount(1)
model = model.fit(rdd)

print ('Zobaczmy jak wyglada wektorowa reprezentacja slow:')
print (model.getVectors())

#Skip-n-Gram - Skip-gram to jedna z technik uczenia nienadzorowanego, wykorzystywanych do znajdowania
    # słów najbardziej powiązanych z danym słowem.

#reprezentacja slowa jest zwrona jako gestwy wektor wielkosci 3
model.transform('ala')

Zobaczmy jak wyglada wektorowa reprezentacja slow:
{'nie': [0.011366038, -0.061218515, 0.07828797, 0.115417056], 'kangura': [0.055424206, -0.06974186, -0.057689942, -0.10136634], 'inne': [-0.012856989, 0.010159543, -0.020994551, 0.075678974], 'jest': [0.00023627638, 0.089690216, -0.024828862, 0.05418939], 'a': [-0.0904789, 0.017946294, -0.012714515, 0.049510144], 'zupelnie': [0.07643746, -0.03218023, -0.1252622, 0.04103435], 'ma': [-0.031219292, 0.10784219, -0.031639963, 0.053703185], 'to': [-0.059488174, -0.021263372, -0.08151385, 0.056854296], 'kota': [0.11185704, -0.093370475, 0.11293637, -0.116649106], 'ale': [0.10859567, 0.10285717, 0.046242457, -0.097419985], 'kot': [-0.113378964, 0.091064535, 0.03890799, 0.076091126], 'ala': [-0.025984691, 0.07186547, 0.041707955, -0.011838384], 'hania': [-0.042351246, 0.03937711, 0.040577114, -0.12101654], 'zdanie': [0.10388712, 0.088408604, -0.11458644, -0.1036376]}


DenseVector([-0.026, 0.0719, 0.0417, -0.0118])

W naszym zabawkowym przykladzie Vec2Model jest zbyt zlozonym modelem. Sprobjmy wykorzystac bardzo prosta reprezetancje wektorowa zliczajaca slowa w dokumencie 

In [6]:
from pyspark.ml.feature import CountVectorizer

cv_model = CountVectorizer(inputCol='tokenized', outputCol='features', 
                            minDF=1.0, vocabSize=100)
cv_model = cv_model.fit(tokenized_docs)
features_df = cv_model.transform(tokenized_docs)

features_df.show()

print (features_df.head())
print (cv_model.vocabulary)
print ('\nPierwszy dokument ma nastepujace zakodowane (w postaci rzadkiej reprezentacji) slowa')
print (features_df.first()['features'])
print ('to jest obiekt sparseVector, ktory reprezentuje wiersz rzadkiej macierzy o 14-tu kolumnach z wartosciami niezerowymi w [0,1,2,3,6,8]')

+--------------------+--------------------+--------------------+
|                text|           tokenized|            features|
+--------------------+--------------------+--------------------+
|Ala ma kota a kot...|[ala, ma, kota, a...|(14,[0,1,2,3,7,11...|
|Hania ma kangura ...|[hania, ma, kangu...|(14,[0,1,3,9,10,1...|
|A to jest zupelni...|[a, to, jest, zup...|(14,[2,4,5,6,8,12...|
+--------------------+--------------------+--------------------+

Row(text='Ala ma kota a kot ma ale', tokenized=['ala', 'ma', 'kota', 'a', 'kot', 'ma', 'ale'], features=SparseVector(14, {0: 2.0, 1: 1.0, 2: 1.0, 3: 1.0, 7: 1.0, 11: 1.0}))
['ma', 'ale', 'a', 'kota', 'to', 'zdanie', 'inne', 'kot', 'jest', 'hania', 'nie', 'ala', 'zupelnie', 'kangura']

Pierwszy dokument ma nastepujace zakodowane (w postaci rzadkiej reprezentacji) slowa
(14,[0,1,2,3,7,11],[2.0,1.0,1.0,1.0,1.0,1.0])
to jest obiekt sparseVector, ktory reprezentuje wiersz rzadkiej macierzy o 14-tu kolumnach z wartosciami niezerowymi w [0,1,2,3

In [7]:
from pyspark.ml.feature import MinHashLSH

mh_model = MinHashLSH(inputCol='features', outputCol='hashes', 
                      numHashTables=2)

#mh_model = MinHashLSH(inputCol='features', outputCol='hashes', numHashTables=6) #"startowo"

mh_model = mh_model.fit(features_df)

similarity = mh_model.approxSimilarityJoin(features_df, features_df, 1, distCol='JaccardDistance')

similarity.show()

+--------------------+--------------------+------------------+
|            datasetA|            datasetB|   JaccardDistance|
+--------------------+--------------------+------------------+
|[A to jest zupeln...|[A to jest zupeln...|               0.0|
|[Hania ma kangura...|[Ala ma kota a ko...|0.6666666666666667|
|[Ala ma kota a ko...|[Hania ma kangura...|0.6666666666666667|
|[Ala ma kota a ko...|[Ala ma kota a ko...|               0.0|
|[Hania ma kangura...|[Hania ma kangura...|               0.0|
+--------------------+--------------------+------------------+



### Zadanie 2
Jaka jest minimalna ilosc fukcji haszujacych aby wychwycic podobienstow 0.666(6) miedzy pierwszym a drugim dokumentem?

In [ ]:
Minimalna ilość fukcji haszujacych, aby
wychwycić podobieństwo 0.(6) między pierwszym,
a drugim dokumentem to: numHashTables=2.
    
Dla numHashTables=1 JaccardDistance daje
jeszcze wszędzie 0.0.

### Zadanie 3
wykorzystaj mh_model.approxNearestNeighbors aby znalezc 'podobny' dokument do new_doc


In [8]:
import pyspark.ml.feature as feature
from pyspark.ml.feature import CountVectorizer

new_doc = "Ala ma psa a nie ma kota"

new_doc_df = spark_session.createDataFrame([(new_doc, )], ["text"])

tokenizer_new = feature.Tokenizer(inputCol="text", outputCol="tokenized")
tokenized_docs_new = tokenizer_new.transform(new_doc_df)
tokenized_docs_new.show()

new_doc_features = cv_model.transform(tokenized_docs_new)

ret = mh_model.approxNearestNeighbors(features_df, new_doc_features.first()['features'],  5) # Przekazując duży
    # zbiór danych i pojedynczy element, znajduje w przybliżeniu co najwyżej k elementów, które mają najbliższą odległość
    # od elementu.

for row in ret.collect():
    print ('Dokument: ' + row['text'] +', podobieństwo: '+ str(row['distCol']))

+--------------------+--------------------+
|                text|           tokenized|
+--------------------+--------------------+
|Ala ma psa a nie ...|[ala, ma, psa, a,...|
+--------------------+--------------------+

Dokument: Ala ma kota a kot ma ale, podobieństwo: 0.4285714285714286
Dokument: Hania ma kangura ale nie ma kota, podobieństwo: 0.625


### Zadanie 4 (duże)

Pobierz zbior danych z rosyjskiej gazety pravda [link](http://fizyka.umk.pl/~mich/pravda_extracted.tar.gz) zebranej podczas ataku Rosji na Ukraine. 
* wczytaj te dokumenty do DataFrame
* zaaplikuj tokenizacje (sprobuje RegexTokenizer)
* usun duplikaty uzywajac MinHashLSH

In [22]:
# Do tokenizacji:
import pyspark.ml.feature as feature
# Do CountVectorizer:
from pyspark.ml.feature import CountVectorizer
# Do MinHASH:
from pyspark.ml.feature import MinHashLSH

# Wczytanie danych:
content = sc.wholeTextFiles("./pravda_extracted/*")

# Utworzenie struktury:
structure = content.collect()
unzipped = []
for entry in structure:
    unzipped.append([entry[1]])

# Wykorzystanie DataFrame (podpunkt 1):
raw = spark_session.createDataFrame(unzipped, ["text"])

# Tokenizacja (podpunkt 2):
tokenizer = feature.RegexTokenizer(inputCol="text",
                                   outputCol="tokenized",
                                   pattern="\\W")
tokenized = tokenizer.transform(raw)
print('Tokenizowany tekst:')
tokenized.show()

# CountVectorizer i CountVectorizerModel mają na celu pomóc w konwersji kolekcji dokumentów tekstowych na
    # wektory liczby tokenów.

# CountVectorizer:
model = CountVectorizer(inputCol='tokenized',
                        outputCol='features')
model = model.fit(tokenized)
features_df = model.transform(tokenized)
print('CountVectorizer:')
features_df.show()

# MinHASH:
minhash_model = MinHashLSH(inputCol='features',
                           outputCol='hashes',
                           numHashTables=2)
minhash_model = minhash_model.fit(features_df)
ret = minhash_model.approxSimilarityJoin(features_df,
                                         features_df,
                                         threshold=1,
                                         distCol='JaccardDistance')

# Współczynnik Jaccarda mierzy podobieństwo między dwoma zbiorami i jest zdefiniowany jako iloraz mocy
    # części wspólnej zbiorów i mocy sumy tych zbiorów.

# Tworzenie zbioru bez duplikatów (podpunkt 3):
clear_data = ret.where("JaccardDistance != 0").orderBy("JaccardDistance") # Zerowa odległość (takie same)
print('Zbiór bez duplikatów:')
clear_data.show()

print('Koniec!')

Tokenizowany tekst:
+--------------------+--------------------+
|                text|           tokenized|
+--------------------+--------------------+
|Nizhny Novgorod p...|[nizhny, novgorod...|
|Can Russia save U...|[can, russia, sav...|
|Women look at eac...|[women, look, at,...|
|Ukrainian border ...|[ukrainian, borde...|
|Anzhi out of Euro...|[anzhi, out, of, ...|
|Syria: Chaos, cor...|[syria, chaos, co...|
|Delegation from L...|[delegation, from...|
|Should zoo animal...|[should, zoo, ani...|
|Europa League: So...|[europa, league, ...|
|Yelena Isinbayeva...|[yelena, isinbaye...|
|Astrakhan city ad...|[astrakhan, city,...|
|The future of Mer...|[the, future, of,...|
|Explosion occurs ...|[explosion, occur...|
|Moscow region mou...|[moscow, region, ...|
|Alive baby nearly...|[alive, baby, nea...|
|Russian troops ta...|[russian, troops,...|
|Ukraine's Yanukov...|[ukraine, s, yanu...|
|Ukrainian parliam...|[ukrainian, parli...|
|Army hazing start...|[army, hazing, st...|
|Longest sus